# Spark session

In [105]:
import findspark
findspark.init()

# import pyspark
# sc = pyspark.SparkContext(appName="AppName")

from pyspark.sql import SparkSession

    # .config("spark.driver.memory", "8g") \
    # .config("spark.executor.memory", "8g") \

spark = SparkSession.builder \
    .appName("PostgreSQL Connection with PySpark") \
    .config("spark.ui.showConsoleProgress", "false") \
    .config("spark.jars", "postgresql-42.6.0.jar") \
    .getOrCreate()

spark.sparkContext.setLogLevel(logLevel='ERROR')


# Spark Postgres connector

In [106]:
PG_DATABASE = 'malisarafi'
PG_USER = 'malisarafiuser'
PG_PASSWORD = 'chJ1L^nokFvga8'
PG_PORT = 5432
PG_HOST = '127.0.0.1'

properties = {
    "user": PG_USER,
    "password": PG_PASSWORD,
    "driver": "org.postgresql.Driver"
}

url = f"jdbc:postgresql://{PG_HOST}:{PG_PORT}/{PG_DATABASE}"

table_name = "news"

df = spark.read.jdbc(url=url, table=table_name, properties=properties)


In [107]:
df.show()

+-----+--------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   id| page_id|                 url|                head|          author|            category|                date|                tags|                text|             summary|
+-----+--------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|29499|14324439|https://www.tgju....|                    |                |                    |                    |                    |                    |                    |
|29503|10874582|https://www.tgju....|                    |                |                    |                    |                    |                    |                    |
|29506|15316541|https://www.tgju....|                    |                |                    

## News statistical info

In [108]:
query = f"(SELECT COUNT(*) FROM {table_name}) AS count"
row_count_df = spark.read \
    .jdbc(url=url, table=query, properties=properties)

# Extract the row count value
row_count = row_count_df.first()[0]
print(row_count)


14943


In [109]:
df.show

<bound method DataFrame.show of DataFrame[id: int, page_id: int, url: string, head: string, author: string, category: string, date: string, tags: string, text: string, summary: string]>

In [110]:
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql.functions import mean, col, desc, count, isnull, isnan, when, rank, sum, trim
import pandas as pd
import matplotlib.pyplot as plt

In [111]:
display(df.dtypes)
df.printSchema()


[('id', 'int'),
 ('page_id', 'int'),
 ('url', 'string'),
 ('head', 'string'),
 ('author', 'string'),
 ('category', 'string'),
 ('date', 'string'),
 ('tags', 'string'),
 ('text', 'string'),
 ('summary', 'string')]

root
 |-- id: integer (nullable = true)
 |-- page_id: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- head: string (nullable = true)
 |-- author: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- text: string (nullable = true)
 |-- summary: string (nullable = true)



In [112]:
df.count()

14943

In [113]:
df.columns

['id',
 'page_id',
 'url',
 'head',
 'author',
 'category',
 'date',
 'tags',
 'text',
 'summary']

In [114]:
len(df.columns)

10

In [115]:
for column in ['author', 'category', 'date']:
    df.select(column).distinct().show()


+--------------------+
|              author|
+--------------------+
|    امیر رضا مصطفایی|
|         ادمین سیستم|
|            پشتیبانی|
|               ادمین|
|             اپراتور|
|نوشته‌های امیر رض...|
|    گروه تولید محتوا|
|نوشته‌های معصومه ...|
|                    |
|         معصومه دانش|
+--------------------+

+--------------------+
|            category|
+--------------------+
| اخبار تولید و تجارت|
|          اخبار ارزی|
|         اخبار خودرو|
|آموزش های ترید و ...|
|بازار ارزهای دیجی...|
|بازار ارزهای دیجی...|
| قوانین بازار رمزارز|
|        اخبار فناوری|
|          اخبار بورس|
|         اخبار بانکی|
|     اخبار طلا و سکه|
|اخبار ارزهای دیجیتال|
|      گزارش‌ بازارها|
|   اخبار نفت و انرژی|
|راهنمای جامع اندی...|
|       برترین برندها|
|          طلا و نقره|
|          اخبار مسکن|
|بازار ارزهای دیجی...|
|                    |
+--------------------+
only showing top 20 rows

+--------------------+
|                date|
+--------------------+
| چهارشنبه 21 دی 1401|
|  شنبه

In [116]:
for column in ['author', 'category', 'date']:
    result = df.groupBy(column).count().orderBy("count", ascending=False)
    display(result.toPandas())


,author,count
0,,13208
1,معصومه دانش,821
2,امیر رضا مصطفایی,533
3,گروه تولید محتوا,162
4,ادمین سیستم,97
5,ادمین,74
6,پشتیبانی,40
7,اپراتور,6
8,نوشته‌های امیر رضا مصطفایی,1
9,نوشته‌های معصومه دانش,1


,category,count
0,,10179
1,اخبار طلا و سکه,892
2,اخبار ارزهای دیجیتال,683
3,اخبار ارزی,511
4,گزارش‌ بازارها,373
5,اخبار نفت و انرژی,259
6,اخبار خودرو,257
7,اخبار بورس,245
8,اخبار فناوری,240
9,اخبار تولید و تجارت,201


,date,count
0,,10531
1,جمعه 13 اسفند 1400,246
2,یکشنبه 25 تیر 1402,229
3,چهارشنبه 28 تیر 1402,216
4,شنبه 24 تیر 1402,215
...,...,...
314,دوشنبه 15 فروردین 1401,1
315,سه شنبه 17 آبان 1401,1
316,چهارشنبه 9 آذر 1401,1
317,دوشنبه 3 مرداد 1401,1


In [117]:
empty_counts = df.select([count(when(trim(df[c]) == '', c)).alias(c) for c in df.columns]).toPandas().T
empty_counts.columns = ['Empty Count']
display(empty_counts)


,Empty Count
id,0
page_id,0
url,0
head,10531
author,13208
category,10179
date,10531
tags,13002
text,10549
summary,10718


In [118]:
for col_name in df.columns:
    df = df.withColumn(col_name, when(col(col_name) == "", None).otherwise(col(col_name)))

df = df.dropna()


In [119]:
df.count()

1597

In [120]:
df.show()

+-----+--------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   id| page_id|                 url|                head|          author|            category|                date|                tags|                text|             summary|
+-----+--------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|32298| 2172793|https://www.tgju....|سناریوی کاهش نفت ...|     معصومه دانش|     اخبار طلا و سکه|دوشنبه 22 خرداد 1402|بهای نفت|نفت خام|...|به گزارششبکه اطلا...|از آنجا که نگرانی...|
|    3|      19|https://www.tgju....|خرس‌های طلا در کمین!|     معصومه دانش|     اخبار طلا و سکه|  یکشنبه 25 تیر 1402|بهای طلا|بهای نقر...|به گزارششبکه اطلا...|گرچه طلا قیمت‌های...|
|    4|      21|https://www.tgju....|چشم‌انداز بیت کوی...|امیر رضا مصطفایی|اخبار ارزهای دیجیتال

# Kafka

In [121]:
# from kafka import KafkaConsumer

# consumer = KafkaConsumer(bootstrap_servers=['localhost:9092']) 
# topics = consumer.topics()

# for topic in topics:
#     print(topic)


In [122]:
from kafka import KafkaAdminClient
client = KafkaAdminClient(bootstrap_servers="localhost:9092")
topics = client.list_topics()

for topic in topics:
    print(topic)

client.delete_topics(topics)


news_from_step3


DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='news_from_step3', error_code=0)])

## Producer

In [123]:
from kafka import KafkaProducer
import os, random
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql.functions import mean, col, desc, count, isnull, isnan, when, rank, sum
import pandas as pd
import matplotlib.pyplot as plt


producer = KafkaProducer(bootstrap_servers='127.0.0.1:9092')

rdd = df.rdd.map(lambda column:
        '\t'.join([str(c) for c in column])
    )

# df=rdd.toDF()
# df.show()
for row in rdd.collect():
    producer.send('news_from_crawler', value=row.encode('utf-8'))

producer.close()

print(f'Sent {rdd.count()} lines in total.')


Sent 1597 lines in total.


## Consumer

In [124]:
from kafka import KafkaConsumer
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, StringType
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import to_timestamp

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("page_id", IntegerType(), True),
    StructField("url", StringType(), True),
    StructField("head", StringType(), True),
    StructField("author", StringType(), True),
    StructField("category", StringType(), True),
    StructField("date", StringType(), True),
    StructField("tags", StringType(), True),
    StructField("text", StringType(), True),
    StructField("summary", StringType(), True),
])

# offset: subscribe from beginning of time
consumer = KafkaConsumer('news_from_crawler', bootstrap_servers='127.0.0.1:9092', auto_offset_reset='earliest')


In [125]:
rows = []
while True:
    partition = ""
    for _, partition in consumer.poll(timeout_ms=5000, max_records=999999).items():
        for msg in partition:
            data = msg.value.decode('utf-8').split('\t')
            rows.append({
                "id": int(data[0]),
                "page_id": int(data[1]),
                "url": data[2],
                "head": data[3],
                "author": data[4],
                "category": data[5],
                "date": data[6],
                "tags": data[7],
                "text": data[8],
                "summary": data[9],
            })
            print(f'\r{len(rows)}', end='')
    if len(partition) == 0:
        break

# to df
df = spark.createDataFrame(rows, schema=schema)


1597

In [126]:
df.show()

+-----+--------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   id| page_id|                 url|                head|          author|            category|                date|                tags|                text|             summary|
+-----+--------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|32298| 2172793|https://www.tgju....|سناریوی کاهش نفت ...|     معصومه دانش|     اخبار طلا و سکه|دوشنبه 22 خرداد 1402|بهای نفت|نفت خام|...|به گزارششبکه اطلا...|از آنجا که نگرانی...|
|    3|      19|https://www.tgju....|خرس‌های طلا در کمین!|     معصومه دانش|     اخبار طلا و سکه|  یکشنبه 25 تیر 1402|بهای طلا|بهای نقر...|به گزارششبکه اطلا...|گرچه طلا قیمت‌های...|
|    4|      21|https://www.tgju....|چشم‌انداز بیت کوی...|امیر رضا مصطفایی|اخبار ارزهای دیجیتال

Close session

In [127]:
spark.sparkContext.stop()
